In [ ]:
# Installing mmcv-full
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.13/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.13/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
# Cloning and installing MMDetection (version 2.x)
!git clone --branch 2.x https://github.com/open-mmlab/mmdetection.git
!cd mmdetection; python setup.py install

Cloning into 'mmdetection'...
remote: Enumerating objects: 38019, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 38019 (delta 39), reused 60 (delta 25), pack-reused 37917
Receiving objects: 100% (38019/38019), 63.24 MiB | 22.31 MiB/s, done.
Resolving deltas: 100% (26201/26201), done.
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packa

In [ ]:
!pip install mmdet

In [ ]:
# Cloning the voc2coco utility
!git clone https://github.com/yukkyo/voc2coco.git # voc -> cooo

Cloning into 'voc2coco'...
remote: Enumerating objects: 436, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 436 (delta 19), reused 11 (delta 11), pack-reused 408
Receiving objects: 100% (436/436), 214.06 KiB | 1.12 MiB/s, done.
Resolving deltas: 100% (390/390), done.


In [ ]:
# Load the configuration file
config_file = './mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = './mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

In [ ]:
!cd ./mmdetection; mkdir checkpoints
!wget -O ./mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

--2024-04-02 18:39:42--  http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.20.226, 47.246.20.229, 47.246.20.218, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.20.226|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167287506 (160M) [application/octet-stream]
Saving to: ‘./mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’

./mmdetection/check 100%[===================>] 159.54M  10.5MB/s    in 16s     

2024-04-02 18:39:59 (10.1 MB/s) - ‘./mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’ saved [167287506/167287506]



In [ ]:
# Configuring the dataset, model, and training parameters
from mmcv import Config

/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [ ]:
!ls -lia ./mmdetection/checkpoints

total 163376
4854574 drwxr-xr-x  2 root root      4096 Apr  2 18:39 .
4849666 drwxr-xr-x 19 root root      4096 Apr  2 18:39 ..
4854575 -rw-r--r--  1 root root 167287506 Nov  2  2021 faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth


In [ ]:
cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [ ]:
cfg.model.roi_head.bbox_head.num_classes = 6  # Number of classes is set to 6

# Loading the pretrained model
cfg.load_from = '/content/latest.pth'  # Path to my trained model

In [ ]:
!mkdir -p /content/data

In [ ]:
!unzip /content/data/blood.zip -d /content/data/

Archive:  /content/data/blood.zip
   creating: /content/data/blood/hard_cases/
   creating: /content/data/blood/hard_cases/jpg/
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000005.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000006.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000010.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000014.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000021.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000022.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000023.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000024.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000025.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000026.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000028.jpg  
  inflating: /content/data/blood/hard_cases/jpg/img_hod_000030.jpg  
  inflating: /content/data/blood/hard_cases/

In [ ]:
from mmcv.runner import HOOKS, Hook

# Defining a new hook class for saving the best checkpoint based on the evaluation metric.
@HOOKS.register_module()
class SaveBestCheckpointHook(Hook):
    def __init__(self, out_dir, metric='bbox_mAP_50', save_optimizer=True): #  metric='bbox_mAP
        self.out_dir = out_dir
        self.metric = metric
        self.save_optimizer = save_optimizer
        self.best_score = 0.0

    def after_train_epoch(self, runner):
        # Check and save the best model checkpoint after each training epoch based on the evaluation metric.
        if not self.every_n_epochs(runner, 1):
            return
        from mmcv.runner import save_checkpoint
        if runner.log_buffer.output.get(self.metric, 0) > self.best_score:
            self.best_score = runner.log_buffer.output[self.metric]
            save_checkpoint(runner.model, self.out_dir, optimizer=self.save_optimizer)

KeyError: 'SaveBestCheckpointHook is already registered in hook'

In [ ]:
name = '/rcnn_normal/'

In [ ]:
# Setting up the name for normal case training
name = '/content/data/blood/normal_cases'

In [ ]:
# Importing required libraries
import os
import xml.etree.ElementTree as ET

# Specifying the directory path for annotations
directory_path = name  + '/xml'

In [ ]:
def remove_spaces_from_tags(dir_path):
    for filename in os.listdir(dir_path):
        if filename.lower().endswith('.xml'):
            filepath = os.path.join(dir_path, filename)

            # Parsing the XML file
            tree = ET.parse(filepath)
            root = tree.getroot()

            # Removing spaces from <filename> and <path> tags
            for tag in ['filename', 'path']:
                element = root.find(tag)
                if element is not None:
                    element.text = element.text.replace(" ", "")

            # Saving the changes back to the XML file
            tree.write(filepath)
            print(f"Changed file: {filename}")

# Executing the function to preprocess XML annotations
remove_spaces_from_tags(directory_path)

Changed file: img_hod_000883.xml
Changed file: img_hod_000800.xml
Changed file: img_hod_005292.xml
Changed file: img_hod_000237.xml
Changed file: img_hod_000668.xml
Changed file: img_hod_005124.xml
Changed file: img_hod_000698.xml
Changed file: img_hod_000011.xml
Changed file: img_hod_005152.xml
Changed file: img_hod_004861.xml
Changed file: img_hod_000856.xml
Changed file: img_hod_005258.xml
Changed file: img_hod_000016.xml
Changed file: img_hod_005255.xml
Changed file: img_hod_010311.xml
Changed file: img_hod_000164.xml
Changed file: img_hod_000149.xml
Changed file: img_hod_000622.xml
Changed file: img_hod_004880.xml
Changed file: img_hod_005211.xml
Changed file: img_hod_005043.xml
Changed file: img_hod_010262.xml
Changed file: img_hod_000056.xml
Changed file: img_hod_000210.xml
Changed file: img_hod_004908.xml
Changed file: img_hod_000295.xml
Changed file: img_hod_000825.xml
Changed file: img_hod_000448.xml
Changed file: img_hod_000781.xml
Changed file: img_hod_000088.xml
Changed fi

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

def save_names_to_file(names, file_path):
    # 确保目录存在
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as file:
        for name in names:
            file.write(name + '\n')

# 读取CSV文件
df = pd.read_csv('/content/data/blood/metadata.csv')

# 筛选出类别为blood的行
blood_df = df[df['Category'] == 'blood']

blood_df = blood_df[df['Case Type'] == 'Normal']
print(blood_df)

# 只获取图像名称列
image_names = blood_df['Image Name'].str.replace('.jpg', '')  # 移除.jpg后缀

# 分割数据集，这里的比例需要根据你的情况来设置
train_names, test_names = train_test_split(image_names, test_size=0.2, random_state=42)
train_names, val_names = train_test_split(train_names, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# 替换为你的实际文件路径
save_names_to_file(train_names, '/content/data/blood/normal_cases/txt/train.txt')
save_names_to_file(val_names, '/content/data/blood/normal_cases/txt/validation.txt')
save_names_to_file(test_names, '/content/data/blood/normal_cases/txt/test.txt')



     Category Case Type          Image Name Annotation Name (YOLOv5)  \
4381    blood    Normal  img_hod_000040.jpg       img_hod_000040.txt   
4382    blood    Normal  img_hod_000814.jpg       img_hod_000814.txt   
4383    blood    Normal  img_hod_010546.jpg       img_hod_010546.txt   
4384    blood    Normal  img_hod_000708.jpg       img_hod_000708.txt   
4385    blood    Normal  img_hod_004854.jpg       img_hod_004854.txt   
...       ...       ...                 ...                      ...   
4930    blood    Normal  img_hod_000904.jpg       img_hod_000904.txt   
4931    blood    Normal  img_hod_000953.jpg       img_hod_000953.txt   
4932    blood    Normal  img_hod_000782.jpg       img_hod_000782.txt   
4933    blood    Normal  img_hod_000459.jpg       img_hod_000459.txt   
4934    blood    Normal  img_hod_010145.jpg       img_hod_010145.txt   

     Annotation Name (Faster R-CNN)  \
4381             img_hod_000040.xml   
4382             img_hod_000814.xml   
4383             i

<ipython-input-24-7f8f0681b265>:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  blood_df = blood_df[df['Case Type'] == 'Normal']
<ipython-input-24-7f8f0681b265>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  image_names = blood_df['Image Name'].str.replace('.jpg', '')  # 移除.jpg后缀


In [ ]:
!python /content/voc2coco/voc2coco.py --ann_dir /content/data/blood/normal_cases/xml \
--ann_ids /content/data/blood/normal_cases/txt/train.txt \
--labels /content/data/blood/normal_cases/txt/labels.txt \
--output /content/data/blood/normal_cases/train.json \
--ext xml

!python /content/voc2coco/voc2coco.py --ann_dir /content/data/blood/normal_cases/xml \
--ann_ids /content/data/blood/normal_cases/txt/validation.txt \
--labels /content/data/blood/normal_cases/txt/labels.txt \
--output /content/data/blood/normal_cases/val.json \
--ext xml

!python /content/voc2coco/voc2coco.py --ann_dir /content/data/blood/normal_cases/xml \
--ann_ids /content/data/blood/normal_cases/txt/test.txt \
--labels /content/data/blood/normal_cases/txt/labels.txt \
--output /content/data/blood/normal_cases/test.json \
--ext xml


Start converting !
100% 332/332 [00:00<00:00, 19290.57it/s]
Start converting !
100% 111/111 [00:00<00:00, 17815.32it/s]
Start converting !
100% 111/111 [00:00<00:00, 17841.26it/s]


In [ ]:
!pip uninstall mmdet -y

Found existing installation: mmdet 2.28.2
Uninstalling mmdet-2.28.2:
  Successfully uninstalled mmdet-2.28.2


In [ ]:
!pip install mmdet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.7 MB/s eta 0:00:00


In [ ]:
# Checking the version of PyTorch
import torch
print(torch.__version__)

1.13.0+cu116


In [ ]:
# Downgrading PyTorch to 1.13.0+cu116 for compatibility with MMDetection
!pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 820.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.17.1+cu121
    Uninstalling torchvision-0.17.1+cu121:
      Successfully uninstalled torchvision-0.17.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.0+cu116 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.0+cu116 which is incompatible.
torchtext 0.17.1 requires torc

In [ ]:
!pip uninstall mmdet -y

Found existing installation: mmdet 3.3.0
Uninstalling mmdet-3.3.0:
  Successfully uninstalled mmdet-3.3.0


In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.13/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.13/index.html


In [ ]:
!pip install mmdet==2.28.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.3 MB/s eta 0:00:00


In [15]:
from mmcv import Config
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset
from mmdet.apis import set_random_seed

# Registering a new dataset class based on the CocoDataset class.
@DATASETS.register_module(force=True)
class HOD(CocoDataset):
    CLASSES = ('alcohol', 'insulting_gesture', 'blood', 'cigarette', 'gun', 'knife')

config_file = './mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'

cfg = Config.fromfile(config_file)

# Modifying the dataset-related configurations.
cfg.dataset_type = 'HOD'
cfg.data_root = '/content/data/blood/normal_cases'

# Updating the type, data_root, ann_file, img_prefix configurations for the train, val, and test datasets..
cfg.data.train.type = 'HOD'
cfg.data.train.data_root = '/content/data/blood/normal_cases'
cfg.data.train.ann_file = 'train.json'
cfg.data.train.img_prefix = 'jpg'

cfg.data.val.type = 'HOD'
cfg.data.val.data_root = '/content/data/blood/normal_cases'
cfg.data.val.ann_file = 'val.json'
cfg.data.val.img_prefix = 'jpg'

cfg.data.test.type = 'HOD'
cfg.data.test.data_root = '/content/data/blood/normal_cases'
cfg.data.test.ann_file = 'test.json'
cfg.data.test.img_prefix = 'jpg'

# Updating the number of classes.
cfg.model.roi_head.bbox_head.num_classes = 6
# Pretrained model
cfg.load_from = '/content/latest.pth'

# Setting the directory for saving the logs of training weights.
cfg.work_dir = './tutorial_exps_normal'

# Updating the learning rate configurations.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 2000

# For CocoDataset, the metric should be set to 'bbox' (instead of mAP). Setting it to 'bbox' calculates mAP over a range of IoU thresholds (0.5 to 0.95)
cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 2000
cfg.checkpoint_config.interval = 5
cfg.custom_hooks = [dict(type='SaveBestCheckpointHook', out_dir=cfg.work_dir, metric='bbox_mAP', save_optimizer=True)]

# Due to a bug(?), set the samples_per_gpu to 1 for test dataset evaluation. In the data loader, the batch size is determined by the number of GPUs.
cfg.data.samples_per_gpu = 1

# Due to an error where loading the config twice causes the lr_config's policy to disappear, it's set again here.
cfg.lr_config.policy='step'
# Set the seed for more reproducible results.
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.custom_hooks = [dict(type='SaveBestCheckpointHook', out_dir=cfg.work_dir, metric='bbox_mAP', save_optimizer=True)]

# Addition by Eunkyeom
# Change the evaluation metric since we use a customized dataset.
cfg.device = 'cuda'
cfg.runner.max_epochs = 150

print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [16]:
!mkdir -p ./show_test_output_normal_to_normal # Create a directory to save the results

In [ ]:
cfg.dump('./tutorial_exps_normal/HOD_faster_rcnn_conf_normal.py')

FileNotFoundError: [Errno 2] No such file or directory: './tutorial_exps_normal/HOD_faster_rcnn_conf_normal.py'

In [17]:
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
# Creating the test Dataset and DataLoader.
# Unlike when creating the train dataset, do not wrap build_dataset() in a list.
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        # Must set the samples_per_gpu argument value to 1
        # samples_per_gpu=cfg.data.samples_per_gpu,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

# Must ensure that the 'img' key value is output as a tensor.
next(iter(data_loader))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


{'img_metas': [DataContainer([[{'filename': '/content/data/blood/normal_cases/jpg/img_hod_000320.jpg', 'ori_filename': 'img_hod_000320.jpg', 'ori_shape': (408, 612, 3), 'img_shape': (800, 1200, 3), 'pad_shape': (800, 1216, 3), 'scale_factor': array([1.9607843, 1.9607843, 1.9607843, 1.9607843], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}]])],
 'img': [tensor([[[[ 1.9920,  1.9920,  2.0092,  ...,  0.0000,  0.0000,  0.0000],
            [ 1.9920,  1.9920,  2.0092,  ...,  0.0000,  0.0000,  0.0000],
            [ 1.9920,  1.9920,  2.0092,  ...,  0.0000,  0.0000,  0.0000],
            ...,
            [-1.0733, -1.0904, -1.1247,  ...,  0.0000,  0.0000,  0.0000],
            [-1.0562, -1.0733, -1.1075,  ...,  0.0000,  0.0000,  0.0000],
            [-1.0562, -1.0733, -1.0904,  ...,  0.0000,  0.0000,  0.0000]],
  
           [[ 1.3606,  1.

In [52]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

checkpoint_file = '/content/latest.pth'

# Using the checkpoint saved model file to create a model, using the above updated config.
# Load model
model_ckpt = init_detector(cfg, checkpoint_file, device='cuda:0')

load checkpoint from local path: /content/latest.pth


In [53]:
from mmdet.apis import multi_gpu_test, single_gpu_test
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

model_ckpt = MMDataParallel(model_ckpt, device_ids=[0])

# Calling single_gpu_test() to perform inference on the test dataset. Must set the batch size to 1.
# The inference results are saved as visualized images in the /kaggle/working/show_test_output directory.
# Set the threshold properly to 0.5 !!!
outputs = single_gpu_test(model_ckpt, data_loader, True, './show_test_output_normal_to_normal', 0.5)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 111/111, 4.5 task/s, elapsed: 25s, ETA:     0s

In [55]:
print('Result outputs type:', type(outputs))
print('Number of evaluated files:', len(outputs))
print('Type of the first evaluation result:', type(outputs[0]))
print('Number of CLASSES in the first evaluation result:', len(outputs[0]))
print('Type and shape of CLASS ID 0 in the first evaluation result:', type(outputs[0][0]), outputs[0][0].shape)

Result outputs type: <class 'list'>
Number of evaluated files: 111
Type of the first evaluation result: <class 'list'>
Number of CLASSES in the first evaluation result: 6
Type and shape of CLASS ID 0 in the first evaluation result: <class 'numpy.ndarray'> (0, 5)


In [56]:
print(outputs[0])

[array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([[128.37439  ,  48.85081  , 337.79398  , 265.43787  ,   0.9371765],
       [218.67978  , 122.77161  , 343.5819   , 231.64253  ,   0.9331763]],
      dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32)]


In [57]:
print(outputs)

[[array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([[128.37439  ,  48.85081  , 337.79398  , 265.43787  ,   0.9371765],
       [218.67978  , 122.77161  , 343.5819   , 231.64253  ,   0.9331763]],
      dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32)], [array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([[2.5909439e+02, 1.5373807e+02, 4.0591760e+02, 2.4668454e+02,
        6.5922463e-01],
       [2.4761476e+02, 1.3171684e+02, 4.0689832e+02, 3.1405481e+02,
        6.0560521e-02]], dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32)], [array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([[2.2783583e+02, 3.1295007e+02, 4.0089713e+02, 5.3014526e+02,
        9.9157310e-01],
       [2.2421828e+02, 2.8384061e+02, 4.0496069e+0

In [48]:
def calculate_class_accuracy(outputs, true_labels):
    class_correct = {class_id: (0, 0) for class_id in range(len(outputs[0]))}
    for output, true_label in zip(outputs, true_labels):
        max_confidence = -1
        predicted_class = -1


        for class_id, detections in enumerate(output):
            if len(detections) > 0:
                max_detection = max(detections, key=lambda x: x[4])
                if max_detection[4] > max_confidence:
                    max_confidence = max_detection[4]
                    predicted_class = class_id


        if predicted_class == true_label:
            class_correct[predicted_class] = (class_correct[predicted_class][0] + 1, class_correct[predicted_class][1] + 1)
        else:
            class_correct[true_label] = (class_correct[true_label][0], class_correct[true_label][1] + 1)


    class_accuracy = {class_id: correct / total if total != 0 else 0 for class_id, (correct, total) in class_correct.items()}
    return class_accuracy


true_labels = [2] * len(outputs)


class_accuracy = calculate_class_accuracy(outputs, true_labels)
print("Class-wise Classification Accuracy:")
for class_id, accuracy in class_accuracy.items():
    print(f"Class {class_id}: {accuracy:.4f}")




Class-wise Classification Accuracy:
Class 0: 0.0000
Class 1: 0.0000
Class 2: 0.9910
Class 3: 0.0000
Class 4: 0.0000
Class 5: 0.0000


In [54]:
def classify_as_blood(outputs):
    # 从所有类别的输出中获取标签为2（blood）的置信度
    blood_confidence = 0.0
    for detection in outputs[2]:  # outputs[2] 对应标签为2的检测结果
        if detection[4] > blood_confidence:
            blood_confidence = detection[4]

    # 如果置信度超过阈值，则输出 blood，否则输出 not blood
    if blood_confidence > 0.5:  # 设置一个阈值，例如0.5
        return 'blood'
    else:
        return 'not blood'

print(classify_as_blood(outputs))

IndexError: index 4 is out of bounds for axis 0 with size 0

In [36]:
metric = dataset.evaluate(outputs, metric='bbox', classwise=True) # AP0.05 ~ 0.95
print(metric)


Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.01s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.488
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.881
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.524
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.502
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.495
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.592
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.592
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.592
 Average Recall     (AR) @[ IoU=0.50:0.9

In [23]:
import json
import copy
import numpy as np

# Function to filter out the detection outputs based on confidence thresholds.
def filter_outputs_by_confidence(outputs, confidence_threshold):
    filtered_outputs = []
    for output_per_image in outputs:
        filtered_output_per_image = []
        for class_index, boxes in enumerate(output_per_image):
            if len(boxes) == 0:
                filtered_output_per_image.append(np.array([]))
                continue
            # Filtering the boxes based on the confidence score.
            filtered_boxes = boxes[boxes[:, 4] >= confidence_threshold]
            filtered_output_per_image.append(filtered_boxes)
        filtered_outputs.append(filtered_output_per_image)
    return filtered_outputs

# Calling single_gpu_test only once to obtain the results.
outputs = single_gpu_test(model_ckpt, data_loader, True, './show_test_output_normal_to_normal', 0.5)

best_threshold = 0.0  # Variable to store the threshold with the highest mAP50 value.
best_mAP50 = 0.0  # Variable to store the highest mAP50 value.

for con_fi in [0.1, 0.2, 0.3, 0.4, 0.5]:
    print(con_fi)
    # Filtering the previously obtained outputs.
    filtered_outputs = filter_outputs_by_confidence(copy.deepcopy(outputs), con_fi)
    # Evaluating the metrics for the filtered outputs.
    metric = dataset.evaluate(filtered_outputs, metric='bbox', classwise=True)  # AP0.05 ~ 0.95
    metric_50 = dataset.evaluate(filtered_outputs, metric='bbox', classwise=True, iou_thrs=[0.5])

    # Updating the best mAP50 value and the corresponding threshold if the current mAP50 value is higher.
    current_mAP50 = metric_50['bbox_mAP']
    if current_mAP50 > best_mAP50:
        best_mAP50 = current_mAP50
        best_threshold = con_fi

    print(metric)
    print()
    print(metric_50)
    print("=============================================================================================================")

# Converting the threshold value with the highest mAP50 to a string.
best_threshold_str = str(best_threshold).replace('.', '_')
# Saving the results to a JSON file.
with open(f'results_conf_{best_threshold_str}.json', 'w') as f:
    json.dump({'best_threshold': best_threshold, 'best_mAP50': best_mAP50}, f)

print(f"Best Threshold: {best_threshold}, Best mAP50: {best_mAP50}")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 111/111, 4.7 task/s, elapsed: 24s, ETA:     0s0.1

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.01s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.487
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.876
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.524
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.502
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.492
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.589
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.589
 Average Recall   

In [34]:
print(correct_predictions)

0


In [39]:
# 计算每个类别的分类准确率
def calculate_class_accuracy(outputs, true_labels):
    class_correct = {class_id: 0 for class_id in range(len(outputs[0]))}
    class_total = {class_id: 0 for class_id in range(len(outputs[0]))}

    for output, true_label in zip(outputs, true_labels):
        # 检查输出是否为空
        if len(output) == 0:
            continue

        for class_id, prediction in enumerate(output):
            # 检查预测结果是否为空
            if len(prediction) == 0:
                continue

            if np.argmax(prediction) == true_label:
                class_correct[class_id] += 1
            class_total[class_id] += 1

    # 计算分类准确率
    class_accuracy = {class_id: correct / total if total != 0 else 0 for class_id, (correct, total) in class_correct.items()}
    return class_accuracy

# 模拟每张图片的真实标签都是2的情况
true_labels = [2] * len(outputs)

# 计算每个类别的分类准确率
class_accuracy = calculate_class_accuracy(outputs, true_labels)
print("Class-wise Classification Accuracy:")
for class_id, accuracy in class_accuracy.items():
    print(f"Class {class_id}: {accuracy}")


TypeError: cannot unpack non-iterable int object

In [28]:
accuracy = correct_predictions / 111
print("Classification Accuracy:", accuracy)

Classification Accuracy: 2.0
